<a href="https://colab.research.google.com/github/jyporse/YOLO/blob/main/YOLO_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/Gdrive')

In [ ]:
!/usr/local/cuda/bin/nvcc --version
## 현재 CUDA버전 확인 현재 11.1
''' NVIDIA 홈페이지에서 확인
https://developer.nvidia.com/rdp/cudnn-download
회원가입 후 다운로드
cuDNN : CUDA의 소프트웨어
deep neural networks를 사용하기 위해 
자신의 CUDA 버전과 맞는 버전 다운
'''
!arch 
# 리눅스 버전확인
'''
다운받은 파일을 구글 Drive의 darknet이란 폴더를 만들고 그 안에 cuDNN 폴더를 만들어서 옮겨놓는다.
연결된 경로복사 후 압축해제
'''
!tar -xzvf Gdrive/MyDrive/Kobot/YOLO/darknet/cuDNN/cudnn-11.1-linux-x64-v8.0.5.39.tgz -C /usr/local/
## 압축해제
!chmod a+r /usr/local/cuda/include/cudnn.h
!cat /usr/local/cuda/include/cudnn.h
# 설치확인

## Load Draknet

In [ ]:
import os
if not os.path.exists('darknet'):
    os.makedirs('darknet')
%cd darknet
%ls

In [ ]:
!ls -la '/content/Gdrive/MyDrive/Kobot/YOLO/darknet/bin/darknet'
!cp -r /content/Gdrive/MyDrive/Kobot/YOLO/darknet/bin/darknet ./darknet
!chmod +x ./darknet

In [ ]:
#YOLO data 가져오기
!cp -r /content/Gdrive/MyDrive/Kobot/YOLO/darknet/weights .
!cp -r /content/Gdrive/MyDrive/Kobot/YOLO/darknet/cfg .
!cp -ar /content/Gdrive/MyDrive/Kobot/YOLO/darknet/data .
%ls

In [ ]:
#download files

def imShow(path):
    import cv2
    import matplotlib.pyplot as plt
    %matplotlib inline

    img = cv2.imread(path)
    height , width = img.shape[:2]
    resized_img = cv2.resize(img, (3*width, 3*height),interpolation = cv2.INTER_CUBIC)

    fig = plt.gcf()
    fig.set_size_inches(18,10)
    plt.axis("off")
    #plt.rcParams['figure.figsize'] = [10,5]
    plt.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
    plt.show()

def upload():
    from google.colab import files
    uploaded = files.upload()
    for name, data in uploaded.items():
        with open(name, "wb") as f:
            f.write(data)
            print("saved file", name)
def download(path):
    from google.colab import files
    files.download(path)

In [ ]:
%ls

In [ ]:
!./darknet detect cfg/yolov3.cfg weights/yolov3.weights data/fruit10.jpg
imShow('predictions.jpg')

### Custom Yolo

In [ ]:
!ls

In [ ]:
!cp -r '/content/Gdrive/MyDrive/Kobot/YOLO/darknet/custom' .
# Custom data가 있는 폴더 복사
!cp -r '/content/Gdrive/MyDrive/Kobot/YOLO/darknet/backup' .

In [ ]:
##!!!!!!!!!!!!!!!!!!!!!! 학습하는 코드 시간이 오래걸림!!!!!!!!!!!!!!!!!!!!!!
!./darknet detector train custom/custom_data.data custom/custom-train-yolo.cfg weights/darknet53.conv.74 -dont-show
## cumstom_data.data 파일의 경로확인
## train, test 경로 확인필요

In [ ]:
!./darknet detector map custom/custom_data.data custom/custom-train-yolo.cfg backup/custom-train-yolo_final.weights
## map를 측정해주는 코드 최적의 final을 찾아야 함

In [ ]:
if os.path.exists('./backup'):
    !cp -r ./backup "/content/Gdrive/MyDrive/Kobot/YOLO/darknet"

if not os.path.exists('./backup'):
    os.makedirs('./backup')
    !cp -r "/content/Gdrive/MyDrive/Kobot/YOLO/darknet/custom-train-yolo_final.weights" ./backup

In [ ]:
!./darknet detector test custom/custom_data.data custom/custom-train-yolo.cfg backup/custom-train-yolo_final.weights data/fruit10.jpg -dont-show

In [ ]:
imShow("predictions.jpg")

### Python을 이용하여 만들기 

In [ ]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
min_confidence = 0.5 
width = 800
height = 0
show_ratio = 1.0
file_name = '/content/Gdrive/MyDrive/Kobot/YOLO/darknet/custom/fruit04.jpg'

net = cv2.dnn.readNet("/content/Gdrive/MyDrive/Kobot/YOLO/darknet/backup/custom-train-yolo_final.weights", "/content/Gdrive/MyDrive/Kobot/YOLO/darknet/custom/custom-train-yolo.cfg")
classes = []
with open("/content/Gdrive/MyDrive/Kobot/YOLO/darknet/custom/classes.names" ,"r") as f:
    classes = [line.strip() for line in f.readlines()]
print(classes)
color_lists = np.random.uniform(0,255, size=(len(classes), 3))

layer_names = net.getLayerNames()
#print(layer_names)
output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
#print(net.getUnconnectedOutLayers)
print(output_layers)

start_time = time.time()
img = cv2.imread(file_name)
h,w = img.shape[:2]
height = int (h* width/ w)
print(height, width) 

blob = cv2.dnn.blobFromImage(img, 0.00392,(416,416), swapRB= True, crop =False)

net.setInput(blob)
outs = net.forward(output_layers)
confidences = []
names = []
boxes = []
colors = []

for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > min_confidence:
            print(detection)
            #Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            y = int(detection[3] * height)
            # Rectangle_coordinates
            x = int(center_x -w/2)
            y = int(center_y - h/2)

            boxes.append([x,y,w,h])
            confidences.appendA(float(confidence))
            names.append(classes[class_id])
            colors.append(color_lists[class_id])

#중복 제거
indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)
font = cv2.FONT_ITALIC
for i in range(len(boxes)):
    if i in indexes:
        x,y,w,h = boxes[i]
        label = '{} {:,.2%}'.format(names[i], confidences[i])
        color = colors[i]
        print(i, label,color, x,y,w,h)
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        cv2.putText(img, label, (x,y-10), font, 1, color,2)
